웹 브라우저로 접근할 수 있는 페이지에 있는 정보 중, 각자가 원하는 정보를 선택하여 로컬 컴퓨터로 가져오는 과정을 스크래핑 (scraping)이라 하며, 크롤링 (crawling)이라고 불리고 있으나, 크롤링은 정확히는 스크래핑과 차이가 있는 의미입니다. 

[참고](http://stackoverflow.com/questions/4327392/what-is-the-difference-between-web-crawling-and-web-scraping)

스크래핑을 하기 위하여 beautiful soup 4라는 HTML parser와 beautiful soup 4가 이용하는 lxml 이라는 XML parser를 이용합니다. 각자의 가상 환경에 해당 패키지가 없다면 아래의 명령어를 통하여 패키지를 설치할 수 있습니다. 

    pip install bs4
    
    pip install lxml

이 외에도 다양한 스크래핑 도구들은 있습니다. 

또한 HTTP를 통하여 웹서버와 통신하기 위한 requests라는 패키지를 이용합니다. 이 역시 각자의 가상환경에 설치가 되어 있지 않다면 pip install requests를 하면 됩니다. anaconda 기본 패키지에는 위 세 도구 모두 설치되어 있습니다. 

    pip install requests

## 웹페이지 탐색

네이버 영화에서 각 영화들을 클릭해보면 url에 공통된 부분이 있습니다. 

    'http://movie.naver.com/movie/bi/mi/basic.nhn?code=134963' # 라라랜드
    'http://movie.naver.com/movie/bi/mi/basic.nhn?code=126034' # 그래이트 워
    'http://movie.naver.com/movie/bi/mi/basic.nhn?code=127382' # 조작된 도시
    
영화 아이디와 url의 공통된 부분을 합치면 각 영화에 해당하는 영화 url을 얻을 수 있습니다. 그렇기 때문에 url을 base와 id 부분으로 나눠서 만들며, 영화 아이디를 넣을 부분을 %s로 표시합니다. 

In [1]:
from bs4 import BeautifulSoup
import os
import re
import requests
import sys
from pprint import pprint

url_basic_base = 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=%s'

라라랜드 영화 아이디는 movie_id = 134963 입니다. 라라랜드 영화와 관련된 메타 데이터를 수집해 봅시다. 

In [2]:
movie_id = 134963 # LaLa Land

## 웹페이지 가져오기

requests는 네이버 영화 서버에 어떤 것을 요청할 수 있는 라이브러리입니다. requests의 requests.get(url)을 하면 해당 url의 서버로부터 웹페이지의 정보들을 얻어옵니다.
requests.get(url)의 return은 텍스트 외에도 header와 같은 많은 정보들을 포함합니다. 이 중에서 우리가 필요한 것은 text (html source)이기 때문에 requests.get(url).text를 html로 저장합니다. html의 type은 str입니다.

BeautifulSoup(html, 'lxml')은 스트링 형식의 html을 lxml이라는 XML parser를 이용하여 문서를 구조화합니다.
웹페이지는 매우 긴 소스 코드로 이뤄져 있습니다. 브라우저 (크롬, 익스프롤러 등)는 이러한 복잡한 소스코드를 잘 구조화하여 화면에 보여주는 프로그램입니다. 매우 긴 코드지만 HTML은 구조화가 잘 되어 있습니다. 역으로 이 구조를 잘 파악하면 우리가 원하는 정보를 손쉽게 가져올 수 있는 것이죠. BeautifulSoup(html, 'lxml')을 한 번 실행함으로써 이미 HTML 문서는 다 구조화 되었습니다. 이제부터는 그 구조화된 문서로부터 우리가 원하는 정보를 가져올 것입니다.

In [3]:
url = url_basic_base % movie_id

r = requests.get(url)
html = r.text
page = BeautifulSoup(html, 'lxml')

requests.get(url)의 결과물의 headers를 살펴보면, 해당 통신과 관련된 메타 정보들이 포함됨을 볼 수 있습니다. 

In [4]:
r.headers

{'Content-Type': 'text/html;charset=UTF-8', 'Via': '1.1 varnish', 'Age': '0', 'Pragma': 'no-cache', 'P3P': 'CP="ALL CURa ADMa DEVa TAIa OUR BUS IND PHY ONL UNI PUR FIN COM NAV INT DEM CNT STA POL HEA PRE LOC OTC", CP="ALL CURa ADMa DEVa TAIa OUR BUS IND PHY ONL UNI PUR FIN COM NAV INT DEM CNT STA POL HEA PRE LOC OTC"', 'Content-Length': '37628', 'Date': 'Fri, 25 Aug 2017 15:48:16 GMT', 'X-Varnish-Cache': 'MISS', 'Server': 'Apache', 'Content-Encoding': 'gzip', 'Cache-Control': 'no-cache, no-store', 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'X-Varnish': '3144830009', 'Accept-Ranges': 'bytes', 'Content-Language': 'ko-KR', 'connection': 'close', 'Vary': 'Accept-Encoding'}

## 영화 제목 가져오기

영화의 영어 제목을 가져와 봅시다. 

![lalaland_1.png](lalaland_1.png)

좌측 상단에 영화 이름이 있습니다. 원하는 정보를 드래그한 뒤, 크롬의 Inspect (한글은 요소 탐색)을 눌러보시면 해당 부분의 source code가 우측에 하이라이팅 되어 나타납니다. La La Land라는 영화 제목은 strong이라는 태그 안에 들어있으며, 그 태그의 class는 h_movie2입니다. HTML에서 태그라는 것은 "<>"으로 시작하여 "</ >"으로 끝나는 부분입니다. 링크의 경우에는 "\<a>"로 시작하여 "\</a>"로 끝납니다. 

"\<strong class=h_movie2">"는 "\<div class=mv_info>"아래에 있다는 것도 볼 수 있습니다. 

    page.select('div[class=mv_info] strong[class=h_movie2]')

위 코드는 mv_info라는 클래스 이름을 갖는 div 아래에 속한, class 이름이 h_movie2인 strong이라는 것을 찾아서 가져온다는 의미입니다. 

![lalaland_2.png](lalaland_2.png)

select의 결과는 하나가 아닐 수 있기 때문에 return type은 list입니다. 실제 우리의 데이터에서도 select에 해당하는 부분이 2개가 있었네요. 그 중 첫번째 부분만을 이용하겠습니다.

In [5]:
title = page.select('div[class=mv_info] strong[class=h_movie2]')
print(type(title))
print(len(title), '\n')

<class 'list'>
2 



In [6]:
print(title, '\n')

[<strong class="h_movie2" title="La La Land										, 					2016">La La Land
					
					, 
					2016</strong>, <strong class="h_movie2" title="La La Land, 2016">La La Land, 2016</strong>] 



In [7]:
print(title[0], '\n')

<strong class="h_movie2" title="La La Land										, 					2016">La La Land
					
					, 
					2016</strong> 



In [8]:
print(title[1], '\n')

<strong class="h_movie2" title="La La Land, 2016">La La Land, 2016</strong> 



BeautifulSoup.select()의 return type은 bs4에서 만들어둔 클래스입니다. 

In [9]:
print(type(title[0]))

<class 'bs4.element.Tag'>


bs4.element.Tag 안에는 HTML tag의 attribute 종류나 값과 같은 태그 정보를 가져오거나 텍스트 부분을 가져올 수 있는 기능이 있습니다. 영화 제목 La La Land, 2016은 텍스트 부분에 있으니 이를 가져오겠습니다. 

In [10]:
title[0].text

'La La Land\r\n\t\t\t\t\t\r\n\t\t\t\t\t, \r\n\t\t\t\t\t2016'

\r\n 과 같은 줄바꿈 기호나 \t과 같은 탭, 띄어쓰기 때문에 텍스트가 깔끔해 보이지 않습니다. 이를 제거하여 깔끔한 영화 제목을 가져옵니다. 

In [11]:
title[0].text.replace("\t", '').replace('\r', '').replace('\n', '')

'La La Land, 2016'

비슷하게 한국어 영화 제목도 가져와봅니다. 

In [12]:
title = page.select('div[class=mv_info] h3[class=h_movie] a')
title[0].text

'라라랜드'

네이버 영화 페이지는 모든 영화에 대하여 각 영화에 해당하는 페이지의 내용만 바뀌며, 그 형식은 일정합니다. 즉 템플릿이 존재하는 웹페이지에서 스크래핑을 할 때는 해당 웹페이지들의 구조를 파악하면 됩니다. 

## try - except를 통한 에러 방지

for loop을 돌면서 여러 영화의 메타 정보를 가져오겠습니다. 그런데 중간에 Exception이 날 수 있습니다. 인터넷이 끊길 수도 있고, 형식이 잘 맞지 않는 HTML이 있을 수도 있습니다. 이 때 한 번 오류가 나면 프로그램이 멈출텐데, 오류가 나는 영화는 건너띄고 다음 영화의 정보를 얻어오고 싶다면 try - except 구문을 이용하면 됩니다

아래의 코드는 i가 3일때 3앞에 a라는 문자를 붙여서 출력하는 코드입니다. 그리고 그 아래 코드는 i가 3일때 a를 붙인 뒤, j를 다시 integer로 casting하는 코드입니다. 그렇다면 i=3 일 때에는 a3을 인티저로 캐스팅 하지 못하여 오류가 납니다. 그리고 그 다음 i=4 일때는 실행이 되지 않습니다.

In [13]:
for i in range(5):    
    
    s = str(i) if i != 3 else 'a%d' % i
    print(s)

0
1
2
a3
4


In [14]:
for i in range(5):    
    
    s = str(i) if i != 3 else 'a%d' % i    
    j = int(s)
    
    print('s = %s, j = %d' % (s, j))

s = 0, j = 0
s = 1, j = 1
s = 2, j = 2


ValueError: invalid literal for int() with base 10: 'a3'

오류가 날 수 있는 부분을 try - except로 감싸면 오류가 난 경우 print(e)에 의하여 오류 형태를 출력해주고 다음 for loop (i=4) 일때로 넘어가게 됩니다. Exception을 e라는 이름으로 받은 뒤 except 안에서 출력하면 해당 예외가 무엇이었는지 알 수 있습니다

In [15]:
errors = []

for i in range(5):    
    try:
        s = str(i) if i != 3 else 'a%d' % i    
        j = int(s)

        print('s = %s, j = %d' % (s, j))
    except Exception as e:
        print(e)
        continue

s = 0, j = 0
s = 1, j = 1
s = 2, j = 2
invalid literal for int() with base 10: 'a3'
s = 4, j = 4


## 여러 영화에 대하여 영화마다 제목 가져오기

기능별로 함수화를 하면 코드가 가독성이 좋아집니다

In [16]:
def get_soup(url):
    try:
        r = requests.get(url).text
        return BeautifulSoup(r, 'lxml')
    except Exception as e:
        exc_type, exc_value, exc_traceback = sys.exc_info()
        traceback_details = {
                     'filename': exc_traceback.tb_frame.f_code.co_filename,
                     'lineno'  : exc_traceback.tb_lineno,
                     'name'    : exc_traceback.tb_frame.f_code.co_name,
                     'type'    : exc_type.__name__,
                     'message' : str(e)
                    }
        pprint(traceback_details)
        return ''

def _parse_title(page):
    try: 
        return page.select('div[class=mv_info] h3[class=h_movie] a')[0].text.strip()
    except:
        return ''

range는 range(b, e)에 대하여 b부터 e까지의 숫자를 1씩 증가시키며 yield (return과 비슷) 합니다. 

In [17]:
for movie_id in range(134960, 134965):
    url = url_basic_base % movie_id
    page = get_soup(url)
    title = _parse_title(page)
    print('%d: %s' % (movie_id, title))

134960: 그놈이다
134961: 어 라 말라
134962: 아이
134963: 라라랜드
134964: 콜 포 헬프


range(b, e, s)를 하면 b 부터 e 까지 s 간격으로 출력됩니다

In [18]:
list(range(1, 6, 2))

[1, 3, 5]

## Packing

코드를 짤 때, 기능별로 함수를 나눠서 적어두면 좋습니다. parse_basic_page라는 함수를 보면, 제목을 가져오는 부분, 장르를 가져오는 부분 등을 나눠서 적어두었습니다. 가독성을 높여주며, 코드에 오류가 있을 때 수정하기가 용이해집니다

In [19]:
# Basic pages
def get_basic_page(movie_id):
    url = url_basic_base % movie_id
    return get_soup(url)

def parse_basic_page(page):
    movie = {}

    score = _parse_main_score(page)
    movie['expert_score'] = score[0]
    movie['netizen_score'] = score[1]

    movie['title'] = _parse_title(page)
    movie['e_title'] = _parse_e_title(page)

    try:
        basic_inf = page.select('dl[class=info_spec]')[0]
        movie['genres'] = _parse_genres(page)
        movie['countries'] = _parse_countries(page)
        movie['running_time'] = _parse_running_time(page)
        movie['open_dates'] = _parse_open_date(page)
        movie['grade'] = _parse_grade(page)
        return movie
    except Exception as e:
        exc_type, exc_value, exc_traceback = sys.exc_info()
        traceback_details = {
                     'filename': exc_traceback.tb_frame.f_code.co_filename,
                     'lineno'  : exc_traceback.tb_lineno,
                     'name'    : exc_traceback.tb_frame.f_code.co_name,
                     'type'    : exc_type.__name__,
                     'message' : str(e)
                    }
        return movie
    
def _parse_title(page):
    try: 
        return page.select('div[class=mv_info] h3[class=h_movie] a')[0].text.strip()
    except:
        return ''

def _parse_e_title(page):
    try:
        return page.select('div[class=mv_info] strong[class=h_movie2]')[0].text.replace('\r', '').replace('\t', '').replace('\n', '').strip()
    except:
        return ''

def _parse_genres(page):
    genres = page.select('a[href^=/movie/sdb/browsing/bmovie.nhn?genre=]')
    return list({genre.text for genre in genres})
    
def _parse_countries(page):
    countries = page.select('a[href^=/movie/sdb/browsing/bmovie.nhn?nation=]')
    return list({country.text for country in countries})
    
def _parse_running_time(page):
    running_time = 0
    try:
        running_time = re.search(r"\d+분", page.text).group()[:-1]
    except:
        running_time = 0
    return running_time

def _parse_open_date(page):
    return list({d for d in re.findall(r"\d+\.\d+\.\d+ 재*개봉", page.text)})

def _parse_grade(page):
    try:
        return page.select('a[href^=/movie/sdb/browsing/bmovie.nhn?grade]')[0].text
    except:
        return ''

def _parse_main_score(page):
    try:
        main_score = page.select('div[class=main_score]')[0]
        expert_score = main_score.select('div[class=spc_score_area] div[class=star_score]')[0].text.replace('\n','')
        netizen_score = main_score.select('div[class=score] div[class=star_score] span[class=st_off]')[0].text.replace('관람객 평점 ', '').replace('점', '')
        return expert_score, netizen_score
    except Exception as e:
        # print('error from _parse_main_score', e)
        return -1, -1

In [20]:
movie_id = 134963
url = url_basic_base % movie_id
page = get_soup(url)
parse_basic_page(page)

{'countries': ['미국'],
 'e_title': 'La La Land, 2016',
 'expert_score': '8.34',
 'genres': ['뮤지컬', '드라마', '멜로/로맨스'],
 'grade': '12세 관람가',
 'netizen_score': '8.87',
 'open_dates': [],
 'running_time': '127',
 'title': '라라랜드'}

하지만 이런 작업들은 네이버 영화 서버의 입장에서는 디도스 공격과 다르지 않습니다. 알지 못하는 컴퓨터에서 비정상적으로 많은 requests를 요청하는 것이기 때문입니다. 가끔씩 어떤 서버들은 이러한 스크래핑 작업을 공격으로 오인하여 아이피를 차단하기도 합니다. 이를 막기 위해서는 적당히 쉬엄쉬엄 크롤링을 하는게 좋습니다. for loop을 돌면서 프로그램을 원하는 만큼 쉴 수 있습니다. 아래 코드는 for loop을 돌며 한 번 숫자를 출력한 뒤, 1.0초를 쉬어주는 것입니다. 영화 제목을 긁을 때에도 영화마다 어느 정도 시간을 주며 쉬어주는 것 (sleep)이 좋습니다.

In [21]:
import time

for i in range(5):
    print(i)
    time.sleep(1.0)
    
for movie_id in range(134960, 134965):
    url = url_basic_base % movie_id
    page = get_soup(url)
    title = _parse_title(page)
    print('%d: %s' % (movie_id, title))
    time.sleep(1.0)

0
1
2
3
4
134960: 그놈이다
134961: 어 라 말라
134962: 아이
134963: 라라랜드
134964: 콜 포 헬프


## User agent 설정

또한 네이버 영화 서버의 입장에서, 위의 코든느 자신의 정보를 제공하지 않은 체, 무명으로 접근하는 프로그램으로 인식할 수 있습니다. 어떤 서버들은 (예, IMDB) 이러한 요청에 대해서는 답변을 주지 않습니다. 이를 해결하기 위하여 requests를 보낼 때, 자신이 누구인지에 대한 정보를 넣어주면 좋습니다. user-agent를 header에 넣어서 requests를 보낼 수 있습니다. 

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

    response = requests.get(url, headers=headers)

위와 같이 requests.get을 할 때, headers에 자신에 대한 정보를 적어주면 좋습니다. 어떤 user-agent를 적어야 하는지는 구글에 python requests user agent라는 제목으로 검색을 해보시면 많은 종류가 나올 겁니다. 

아래는 참고한 stackoverflow 주소입니다. fake_useragent라는 라이브러리도 있다고 합니다.

http://stackoverflow.com/questions/27652543/how-to-use-python-requests-to-fake-a-browser-visit

## 파일 다운로드

파이썬에서도 음악/사진 파일들을 다운로드 받을 수 있습니다. 다운로드라는 것도 서버와 내 컴퓨터 간의 통로를 열어두고, 전송이 되는 byte 정보들을 모아서 다시 음악, 사진 포멧으로 읽는 것입니다. 물론 스트리밍 서비스들은 temporal하게 내 컴퓨터에 데이터가 쌓이지 않게 막을 수도 있습니다. 그런 종류가 아니라, \<a>라는 태그로 링크가 걸려있는 이미지 파일들을 다운로드 해보겠습니다. 

아래 코드는 urllib.request.urlopen을 통하여 서버와 내 컴퓨터 간의 통신 통로를 열어둡니다. while loop 안에서 열려진 통로에서 100000000 byte만큼의 정보를 가져와 buffer에 넣습니다. 그리고 이 정보를 미리 열어둔 downloaded_file이라는 파일에 적습니다. 주고 받은 정보가 텍스트가 아니라 바이트이기 때문에 'wb'로 파일을 열어둡니다. 다운로드가 모두 끝나면 열어둔 통신 서버를 닫고
    
    opened.close()
    
열어둔 파일도 닫습니다. 

    downloaded_file.close()
    
다운로드가 성공적으로 되었다면 True가, 실패했다면 False가 return 되도록 try - except로 이 코드 부분을 감싸줍니다

In [22]:
import os
import sys
import urllib

def download_image(url, fname):
    try:
        downloaded_file = open(fname, "wb")
        opened = urllib.request.urlopen(url)
        while True:
            buffer = opened.read(100000000)
            if len(buffer) == 0:
                break
            downloaded_file.write(buffer)
        downloaded_file.close()
        opened.close()
        return True
    except:
        return False


만든 함수로 google logo를 다운로드 받아봅니다. 

In [23]:
download_image('https://www.google.co.kr/images/branding/googlelogo/2x/googlelogo_color_272x92dp.png', 'google_logo.png')

True

다운로드한 이미지는 아래와 같습니다. 

![google_logo](google_logo.png)